# Updated Data to Test with

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #Increasing width of Jupyter Notebooks to be able to read this better.

In [4]:
import pandas as pd #Importing pandas to be able to access/clean data in dataframes
import re #Importing re to be able to use regular expressions to clean data
import seaborn as sns
import matplotlib.pyplot as plt


## Reading, Cleaning and Transforming Covid-19 Data from Dr. Hoover (Included in Final Model)

In [5]:
# 1. Reading in covid csv file into dataframe (update this often by changing date in the link)
covid = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/07-25-2020.csv') #First dataset timestamp is 7/9/2020
#covid[(covid['Province_State'] =='Texas')]
covid

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-07-26 04:35:00,34.223334,-82.461707,264,4,0,260.0,"Abbeville, South Carolina, US",1076.364822,1.515152
1,22001.0,Acadia,Louisiana,US,2020-07-26 04:35:00,30.295065,-92.414197,2111,62,0,2049.0,"Acadia, Louisiana, US",3402.369248,2.936997
2,51001.0,Accomack,Virginia,US,2020-07-26 04:35:00,37.767072,-75.632346,1060,15,0,1045.0,"Accomack, Virginia, US",3280.108924,1.415094
3,16001.0,Ada,Idaho,US,2020-07-26 04:35:00,43.452658,-116.241552,6732,45,0,6687.0,"Ada, Idaho, US",1397.878265,0.668449
4,19001.0,Adair,Iowa,US,2020-07-26 04:35:00,41.330756,-94.471059,20,0,0,20.0,"Adair, Iowa, US",279.642058,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3925,NaN,NaN,Unknown,Colombia,2020-07-17 22:34:48,NaN,NaN,0,0,0,0.0,"Unknown, Colombia",NaN,NaN
3926,2070.0,Dillingham,Alaska,US,2020-07-17 18:35:12,59.796037,-158.238194,1,0,0,1.0,"Dillingham, Alaska, US",20.341741,0.000000
3927,NaN,NaN,Grand Princess,Canada,2020-07-13 12:34:33,NaN,NaN,13,0,13,0.0,"Grand Princess, Canada",NaN,0.000000
3928,16061.0,Lewis,Idaho,US,2020-07-10 02:34:22,46.233153,-116.434146,1,0,0,1.0,"Lewis, Idaho, US",26.055237,0.000000


In [6]:
# 2. Data Cleaning: Covid-19 Data
my_data = covid.drop(columns = ['FIPS', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 'Combined_Key', 'Case-Fatality_Ratio', 'Recovered']) #Dropping unnecessary columns (will be getting FIPS codes for all counties further down in this code, important to note that not all Texas counties have covid data reported (Only 245 counties here when 254 exist in Texas) Researched this further at https://dshs.texas.gov/coronavirus/AdditionalData.aspx and found that these counties just had no confirmed cases marked as 0, will be adressing this later in the code.)
covid_texas = my_data[my_data['Province_State'] =='Texas'] #Filtering data to just state of interest - Texas. 
covid_texas = covid_texas.rename(columns= {'Province_State':'State', 'Admin2':'County'}) #Renaming columns for clarity and so that merges can occur later in the code. 
covid_texas = covid_texas.dropna() #Dropping the unassigned county.
covid_texas = covid_texas.reset_index(drop = True) #Resetting index to make count of counties easier (clearly not 254 counties)/ code cleaner from a calrity standpoint. 
covid_texas

,County,State,Confirmed,Deaths,Active,Incidence_Rate
0,Anderson,Texas,2140,4,2136.0,3706.590456
1,Andrews,Texas,225,4,221.0,1202.886929
2,Angelina,Texas,1405,14,1391.0,1620.250245
3,Aransas,Texas,123,0,123.0,523.181625
4,Archer,Texas,15,0,15.0,175.377061
...,...,...,...,...,...,...
246,Wood,Texas,241,6,235.0,529.216715
247,Yoakum,Texas,63,1,62.0,723.057500
248,Young,Texas,101,1,100.0,560.799556
249,Zapata,Texas,122,1,121.0,860.427393


In [7]:
# 3. Data Transformaiton: Adding necessary column to calculate lethality rate. 
covid_texas['Lethality_Rate'] = pd.Series((covid_texas['Deaths']/covid_texas['Confirmed']*100) , index=covid_texas.index) #Expressed as a whole number percentage
covid_texas

,County,State,Confirmed,Deaths,Active,Incidence_Rate,Lethality_Rate
0,Anderson,Texas,2140,4,2136.0,3706.590456,0.186916
1,Andrews,Texas,225,4,221.0,1202.886929,1.777778
2,Angelina,Texas,1405,14,1391.0,1620.250245,0.996441
3,Aransas,Texas,123,0,123.0,523.181625,0.000000
4,Archer,Texas,15,0,15.0,175.377061,0.000000
...,...,...,...,...,...,...,...
246,Wood,Texas,241,6,235.0,529.216715,2.489627
247,Yoakum,Texas,63,1,62.0,723.057500,1.587302
248,Young,Texas,101,1,100.0,560.799556,0.990099
249,Zapata,Texas,122,1,121.0,860.427393,0.819672


## Reading, Cleaning and Transforming Texas Population and Age Data (Included in Final Model)

In [8]:
# 1. Reading in the Population and Age Data file for Texas into dataframe (from https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html, Dataset: Annual County and Resident Population Estimates by Selected Age Groups and Sex: April 1, 2010 to July 1, 2019 (original filename CC-EST2019-AGESEX) )
covid_age_pop = pd.read_csv('Population and Age Data.csv')
covid_age_pop

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,48,1,Texas,Anderson County,1,58458,35521,22937,3135,...,757,898,349,549,865,298,567,39.1,38.8,39.9
1,50,48,1,Texas,Anderson County,2,58452,35518,22934,3135,...,758,897,349,548,865,298,567,39.1,38.8,39.9
2,50,48,1,Texas,Anderson County,3,58493,35542,22951,3129,...,758,903,358,545,864,301,563,39.1,38.8,39.9
3,50,48,1,Texas,Anderson County,4,58394,35586,22808,3105,...,751,916,368,548,842,294,548,39.1,38.7,40.2
4,50,48,1,Texas,Anderson County,5,58059,35327,22732,2999,...,780,942,389,553,851,302,549,39.2,38.7,40.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3043,50,48,507,Texas,Zavala County,8,12310,6211,6099,1050,...,141,210,79,131,215,89,126,30.4,28.5,32.4
3044,50,48,507,Texas,Zavala County,9,12121,6126,5995,1001,...,136,200,77,123,217,91,126,30.6,28.9,32.5
3045,50,48,507,Texas,Zavala County,10,11957,6051,5906,946,...,141,194,81,113,217,89,128,31.1,29.5,32.9
3046,50,48,507,Texas,Zavala County,11,11969,6051,5918,911,...,154,192,77,115,210,86,124,31.0,29.5,32.7


In [9]:
# 2./# 3. Data Cleaning/Transformaiton: Population and Age Data
covid_age_pop = covid_age_pop.loc[(covid_age_pop['YEAR'] == 12)] #Marks the 7/1/2019 population estimate (most recent estimate) as per the data dictionary found here: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-agesex.pdf
covid_age_pop = covid_age_pop.drop(columns = ['COUNTY', 'SUMLEV', 'STNAME', 'STATE','AGE16PLUS_TOT', 'AGE16PLUS_MALE', 'AGE16PLUS_FEM', 'AGE18PLUS_TOT', 'AGE18PLUS_MALE', 'AGE18PLUS_FEM', 'MEDIAN_AGE_MALE', 'MEDIAN_AGE_FEM', 'UNDER5_MALE','UNDER5_FEM','AGE513_MALE','AGE513_FEM','AGE1417_MALE','AGE1417_FEM','AGE1824_MALE','AGE1824_FEM','AGE16PLUS_MALE','AGE16PLUS_FEM','AGE18PLUS_MALE','AGE18PLUS_FEM','AGE1544_MALE','AGE1544_FEM','AGE2544_MALE','AGE2544_FEM','AGE4564_MALE','AGE4564_FEM','AGE65PLUS_MALE','AGE65PLUS_FEM','AGE04_MALE','AGE04_FEM','AGE59_MALE','AGE59_FEM','AGE1014_MALE','AGE1014_FEM', 'AGE1519_MALE','AGE1519_FEM','AGE2024_MALE','AGE2024_FEM','AGE2529_MALE','AGE2529_FEM','AGE3034_MALE','AGE3034_FEM','AGE3539_MALE','AGE3539_FEM','AGE4044_MALE','AGE4044_FEM','AGE4549_MALE','AGE4549_FEM','AGE5054_MALE','AGE5054_FEM','AGE5559_MALE','AGE5559_FEM','AGE6064_MALE','AGE6064_FEM','AGE6569_MALE','AGE6569_FEM','AGE7074_MALE','AGE7074_FEM','AGE7579_MALE','AGE7579_FEM','AGE8084_MALE','AGE8084_FEM','AGE85PLUS_MALE','AGE85PLUS_FEM', 'UNDER5_TOT','AGE1544_TOT', 'AGE2544_TOT','AGE4564_TOT', 'AGE65PLUS_TOT' ])
#Dropping County Numerical Representation as I will only need the actual name to merge on. 
covid_age_pop = covid_age_pop.rename(columns={'CTYNAME': 'County'})
#Age groups edited as per recommendation for sponsor (after data report turned in)
covid_age_pop['County'] = covid_age_pop['County'].replace(' County', '', regex = True) #Counties have the county name followed by the word county, using regex to remove the ' County' to be able to merge on this column later.
covid_age_pop['Age017_Per_Pop_Est'] = (covid_age_pop['AGE04_TOT'] + covid_age_pop['AGE513_TOT'] + covid_age_pop['AGE1417_TOT'])/covid_age_pop['POPESTIMATE'] #Creating come consistency by looking at age group as a proportion of the population of the county, could always decide to scrap this later. 
covid_age_pop['Age1844_Per_Pop_Est'] = (covid_age_pop['AGE1824_TOT']  + covid_age_pop['AGE2529_TOT'] + covid_age_pop['AGE3034_TOT'] + covid_age_pop['AGE3539_TOT'] + covid_age_pop['AGE4044_TOT'])/covid_age_pop['POPESTIMATE']
covid_age_pop['Age4554_Per_Pop_Est'] = (covid_age_pop['AGE4549_TOT'] + covid_age_pop['AGE5054_TOT'])/covid_age_pop['POPESTIMATE']
covid_age_pop['Age_5564_Per_Pop_Est'] = (covid_age_pop['AGE5559_TOT']  + covid_age_pop['AGE6064_TOT'])/covid_age_pop['POPESTIMATE']
covid_age_pop['Age_65_plus_Per_Pop_Est'] = (covid_age_pop['AGE6569_TOT'] + covid_age_pop['AGE7074_TOT'] + covid_age_pop['AGE7579_TOT'] + covid_age_pop['AGE8084_TOT'] + covid_age_pop['AGE85PLUS_TOT'])/covid_age_pop['POPESTIMATE']
covid_age_pop['Pop_Female_Ratio'] = covid_age_pop['POPEST_FEM']/covid_age_pop['POPESTIMATE'] #Looking at gender as a ratio of population
covid_age_pop['Pop_Male_Ratio'] = covid_age_pop['POPEST_MALE']/covid_age_pop['POPESTIMATE']
covid_age_pop = covid_age_pop.rename(columns={'POPEST_MALE':'Population_Male', 'POPEST_FEM':'Population_Female', 'POPESTIMATE': 'Population_Estimate'}) #Renaming the Ctyname to County so that I can merge (join) on that column.


covid_age_pop = covid_age_pop.drop(columns=['AGE1824_TOT','AGE1417_TOT','AGE513_TOT','YEAR','AGE04_TOT','AGE59_TOT','AGE1014_TOT', 'AGE1519_TOT', 'AGE2024_TOT', 'AGE2529_TOT', 'AGE3034_TOT', 'AGE3539_TOT', 'AGE4044_TOT','AGE4549_TOT', 'AGE5054_TOT','AGE5559_TOT','AGE6064_TOT', 'AGE6569_TOT','AGE7074_TOT', 'AGE7579_TOT','AGE8084_TOT','AGE85PLUS_TOT' ])
#Making larger age buckets to assess age, this may change as our model changes and as new data comes out that warrant other groupings.


covid_age_pop


,County,Population_Estimate,Population_Male,Population_Female,MEDIAN_AGE_TOT,Age017_Per_Pop_Est,Age1844_Per_Pop_Est,Age4554_Per_Pop_Est,Age_5564_Per_Pop_Est,Age_65_plus_Per_Pop_Est,Pop_Female_Ratio,Pop_Male_Ratio
11,Anderson,57735,35474,22261,39.4,0.190179,0.400329,0.142461,0.117069,0.149961,0.385572,0.614428
23,Andrews,18705,9568,9137,31.5,0.310238,0.377653,0.110131,0.103983,0.097995,0.488479,0.511521
35,Angelina,86715,42353,44362,37.5,0.254362,0.333529,0.121651,0.124142,0.166315,0.511584,0.488416
47,Aransas,23510,11633,11877,51.2,0.177329,0.254870,0.114334,0.164611,0.288856,0.505189,0.494811
59,Archer,8553,4186,4367,44.1,0.215480,0.293581,0.130597,0.164270,0.196072,0.510581,0.489419
...,...,...,...,...,...,...,...,...,...,...,...,...
2999,Wood,45539,22602,22937,48.9,0.188564,0.272865,0.109005,0.154483,0.275083,0.503678,0.496322
3011,Yoakum,8713,4434,4279,30.6,0.326983,0.347871,0.097900,0.108688,0.118558,0.491105,0.508895
3023,Young,18010,8942,9068,41.6,0.236591,0.298723,0.112937,0.142199,0.209550,0.503498,0.496502
3035,Zapata,14179,7051,7128,30.0,0.330630,0.345440,0.105367,0.086960,0.131603,0.502715,0.497285


In [10]:
# 1. Extract - Reading in the Population and Race Data file for Texas into dataframe (from https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html, Dataset: Annual County and Resident Population Estimates by Selected Age Groups and Sex: April 1, 2010 to July 1, 2019 (original filename CC-EST2019-AGESEX) )
covid_race_pop = pd.read_csv('Population and Race Data.csv')
covid_race_pop

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE
0,50,48,1,Texas,Anderson County,12,0,57735,35474,22261,...,9322,3306,263,165,325,213,54,42,564,446
1,50,48,3,Texas,Andrews County,12,0,18705,9568,9137,...,212,170,140,134,68,62,3,1,136,138
2,50,48,5,Texas,Angelina County,12,0,86715,42353,44362,...,6453,7142,333,332,446,477,38,27,610,631
3,50,48,7,Texas,Aransas County,12,0,23510,11633,11877,...,247,159,163,145,231,260,12,12,229,211
4,50,48,9,Texas,Archer County,12,0,8553,4186,4367,...,51,45,62,69,16,25,3,3,74,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,50,48,499,Texas,Wood County,12,0,45539,22602,22937,...,1344,1186,226,249,141,138,16,20,366,357
250,50,48,501,Texas,Yoakum County,12,0,8713,4434,4279,...,69,64,88,65,26,27,3,0,53,58
251,50,48,503,Texas,Young County,12,0,18010,8942,9068,...,170,137,125,123,65,76,4,6,134,144
252,50,48,505,Texas,Zapata County,12,0,14179,7051,7128,...,49,29,29,35,14,21,1,1,26,24


In [11]:
# 2./ # 3. Data Cleaning/Transformation: Texas Race Data
#Some columns deleted in excel before import into Jupyter. There were extremely specific columns about race/ethnicity, we chose to keep it as simple as we could and look at counts of a single race.
#Data was grouped by year, filtered in excel to only look at year 12 (most recent data from 7/1/2019), and furthermore was filtered to look at total across all ages, so filtered by agegrp to year 0.
covid_race_pop = covid_race_pop.rename(columns={'CTYNAME': 'County'}) #Changed names to be able to merge on this.
covid_race_pop['County'] = covid_race_pop['County'].replace(' County', '', regex = True) #Counties have the county name followed by County, used regex to remove the ' County' to be able to merge on this column later.

covid_race_pop['White'] = covid_race_pop['WA_MALE'] + covid_race_pop['WA_FEMALE'] #Combining Male and Female data since sex is already data that is included in another dataframe. Made it super easy to revist this if we want. May end up bringing these back if we find it helpful for the model.
covid_race_pop['Black'] = covid_race_pop['BA_MALE'] + covid_race_pop['BA_FEMALE']
covid_race_pop['American_Indian_Alaskan_Native'] = covid_race_pop['IA_MALE'] + covid_race_pop['IA_FEMALE']
covid_race_pop['Asian'] = covid_race_pop['AA_MALE'] + covid_race_pop['AA_FEMALE']
covid_race_pop['Native_Hawaiian_Other_Pacific_Islander'] =covid_race_pop['NA_MALE'] + covid_race_pop['NA_FEMALE']
covid_race_pop['Two_Or_More_Races'] =covid_race_pop['TOM_MALE'] + covid_race_pop['TOM_FEMALE']
covid_race_pop = covid_race_pop.drop(columns=['WA_MALE','WA_FEMALE','BA_MALE','BA_FEMALE','IA_MALE','IA_FEMALE','AA_MALE','AA_FEMALE','NA_MALE','NA_FEMALE','TOM_MALE','TOM_FEMALE'])

covid_race_pop['White_Pop_Ratio'] = covid_race_pop['White']/covid_race_pop['TOT_POP'] #Adding series to get ratios of race of each population to be able to better compare across counties.
covid_race_pop['Black_Pop_Ratio'] = covid_race_pop['Black']/covid_race_pop['TOT_POP']
covid_race_pop['American_Indian_Alaskan_Native_Pop_Ratio'] = covid_race_pop['American_Indian_Alaskan_Native']/covid_race_pop['TOT_POP']
covid_race_pop['Asian_Pop_Ratio'] = covid_race_pop['Asian']/covid_race_pop['TOT_POP']
covid_race_pop['Native_Hawaiian_Other_Pacific_Islander_Pop_Ratio'] = covid_race_pop['Native_Hawaiian_Other_Pacific_Islander']/covid_race_pop['TOT_POP']
covid_race_pop['Two_Or_More_Races_Pop_Ratio'] = covid_race_pop['Two_Or_More_Races']/covid_race_pop['TOT_POP']
covid_race_pop = covid_race_pop.drop(columns = ['YEAR', 'TOT_POP','SUMLEV','STATE','COUNTY','STNAME', 'AGEGRP','TOT_MALE','TOT_FEMALE']) #Dropped unncessary columns

covid_race_pop

,County,White,Black,American_Indian_Alaskan_Native,Asian,Native_Hawaiian_Other_Pacific_Islander,Two_Or_More_Races,White_Pop_Ratio,Black_Pop_Ratio,American_Indian_Alaskan_Native_Pop_Ratio,Asian_Pop_Ratio,Native_Hawaiian_Other_Pacific_Islander_Pop_Ratio,Two_Or_More_Races_Pop_Ratio
0,Anderson,43035,12628,428,538,96,1010,0.745388,0.218723,0.007413,0.009318,0.001663,0.017494
1,Andrews,17641,382,274,130,4,274,0.943117,0.020422,0.014648,0.006950,0.000214,0.014648
2,Angelina,70226,13595,665,923,65,1241,0.809848,0.156778,0.007669,0.010644,0.000750,0.014311
3,Aransas,21841,406,308,491,24,440,0.929009,0.017269,0.013101,0.020885,0.001021,0.018715
4,Archer,8138,96,131,41,6,141,0.951479,0.011224,0.015316,0.004794,0.000702,0.016485
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,41496,2530,475,279,36,723,0.911219,0.055557,0.010431,0.006127,0.000791,0.015877
250,Yoakum,8260,133,153,53,3,111,0.948009,0.015265,0.017560,0.006083,0.000344,0.012740
251,Young,17026,307,248,141,10,278,0.945364,0.017046,0.013770,0.007829,0.000555,0.015436
252,Zapata,13950,78,64,35,2,50,0.983849,0.005501,0.004514,0.002468,0.000141,0.003526


## Reading, Cleaning and Transforming Texas Counties and Sectors Data (Excluded in Final Model but used for FIPS codes)


In [12]:
# 1. Extract - Reading in the Texas Counties and Sectors Excel sheet from (https://dshs.texas.gov/chs/info/info_txco.shtm#:~:text=Texas%20Counties%2C%20Public%20Health%20Regions%2C%20and%20Administrative%20Regions,%20%2011%20%2084%20more%20rows%20 (table in excel format linke))
#Column added in excel to express entire FIPS number (used concatenate)
fips_sector_data = pd.read_excel('Texas_counties_sectors.xlsx')
fips_sector_data

,County Name,FIPS#,FIPS #,County #,Public Health Region (11),Health Service Region (8),Metropolitan Statistical Area (MSA),Metropolitan Divisions (MD),Metro Area (82),NCHS Urban Rural Classification (2006),NCHS Urban Rural Classification (2013),Border 32 (La Paz Agreement),Border 15
0,Anderson,48001.0,1.0,1.0,4.0,4/5N,--,--,Non-Metro,Micropolitan,Micropolitan,Non-Border,Non-Border
1,Andrews,48003.0,3.0,2.0,9.0,9/10,--,--,Non-Metro,Micropolitan,Micropolitan,Non-Border,Non-Border
2,Angelina,48005.0,5.0,3.0,5.0,4/5N,--,--,Non-Metro,Micropolitan,Micropolitan,Non-Border,Non-Border
3,Aransas,48007.0,7.0,4.0,11.0,11,Corpus Christi,--,Metro,Medium Metro,Medium Metro,Non-Border,Non-Border
4,Archer,48009.0,9.0,5.0,2.0,2/3,Wichita Falls,--,Metro,Small Metro,Small Metro,Non-Border,Non-Border
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Young,48503.0,503.0,252.0,2.0,2/3,--,--,Non-Metro,Noncore,Non-core,Non-Border,Non-Border
252,Zapata,48505.0,505.0,253.0,11.0,11,--,--,Non-Metro,Noncore,Micropolitan,Border,Border
253,Zavala,48507.0,507.0,254.0,8.0,8,--,--,Non-Metro,Noncore,Non-core,Border,Non-Border
254,Back to CHS Useful Information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# 2. Data Cleaning: Texas Counties and Sectors
fips_sector_data = fips_sector_data.rename(columns = {'County Name': 'County', 'FIPS#': 'FIPS', 'Public Health Region (11)':'Public_Health_Region','Health Service Region (8)':'Health_Service_Region','Metro Area (82)':'Metro_Area', 'NCHS Urban Rural Classification (2013)':'NCHS_Urban_Rural_Classification_2013' }) #Renaming columns to be able to join on County and to use this data in R without issues.
fips_sector_data = fips_sector_data.drop(columns=['County #','FIPS #', 'Metropolitan Divisions (MD)','NCHS Urban Rural Classification (2006)', 'Border 32 (La Paz Agreement)', 'Border 15', 'Metropolitan Statistical Area (MSA)' ]) #Drop unnecessary data.
fips_sector_data = fips_sector_data.dropna() #Get rid of weird NA values at bottom
fips_sector_data

,County,FIPS,Public_Health_Region,Health_Service_Region,Metro_Area,NCHS_Urban_Rural_Classification_2013
0,Anderson,48001.0,4.0,4/5N,Non-Metro,Micropolitan
1,Andrews,48003.0,9.0,9/10,Non-Metro,Micropolitan
2,Angelina,48005.0,5.0,4/5N,Non-Metro,Micropolitan
3,Aransas,48007.0,11.0,11,Metro,Medium Metro
4,Archer,48009.0,2.0,2/3,Metro,Small Metro
...,...,...,...,...,...,...
249,Wood,48499.0,4.0,4/5N,Non-Metro,Non-core
250,Yoakum,48501.0,1.0,1,Non-Metro,Non-core
251,Young,48503.0,2.0,2/3,Non-Metro,Non-core
252,Zapata,48505.0,11.0,11,Non-Metro,Micropolitan


In [14]:
# 3. Data Transformation: Adding neccessary dummy variables for Metro Area and NCHS Urban/Rural columns
fips_sector_data_dummies = pd.get_dummies(fips_sector_data, columns=['Metro_Area','NCHS_Urban_Rural_Classification_2013'])
fips_sector_data_dummies

,County,FIPS,Public_Health_Region,Health_Service_Region,Metro_Area_Metro,Metro_Area_Non-Metro,NCHS_Urban_Rural_Classification_2013_Large Central Metro,NCHS_Urban_Rural_Classification_2013_Large Fringe Metro,NCHS_Urban_Rural_Classification_2013_Medium Metro,NCHS_Urban_Rural_Classification_2013_Micropolitan,NCHS_Urban_Rural_Classification_2013_Non-core,NCHS_Urban_Rural_Classification_2013_Small Metro
0,Anderson,48001.0,4.0,4/5N,0,1,0,0,0,1,0,0
1,Andrews,48003.0,9.0,9/10,0,1,0,0,0,1,0,0
2,Angelina,48005.0,5.0,4/5N,0,1,0,0,0,1,0,0
3,Aransas,48007.0,11.0,11,1,0,0,0,1,0,0,0
4,Archer,48009.0,2.0,2/3,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,48499.0,4.0,4/5N,0,1,0,0,0,0,1,0
250,Yoakum,48501.0,1.0,1,0,1,0,0,0,0,1,0
251,Young,48503.0,2.0,2/3,0,1,0,0,0,0,1,0
252,Zapata,48505.0,11.0,11,0,1,0,0,0,1,0,0


## Reading, Cleaning and Transforming Texas Social Distancing Score Data (Included in Final Model)

In [15]:
# 1. Extract - Reading in the Covid-19 Social Distancing Score Texas data extracted manually into an xlsx from (https://www.unacast.com/covid19/social-distancing-scoreboard?view=state&fips=48) 
#Tried to scrape this, were having issues, not a huge deal, we can manually grab and update this, current as of 7/26/2020)
social_distancing_scores = pd.read_excel('Covid-19 Social Distancing Scores Texas New Data.xlsx')
social_distancing_scores

,County,Social_Distancing_Score,Rating
0,Brewster County,B-,7.0
1,Fisher County,B-,7.0
2,Winkler County,B-,7.0
3,Zavala County,B-,7.0
4,Throckmorton County,B-,7.0
...,...,...,...
249,Terrell County,NaN,NaN
250,Loving County,NaN,NaN
251,Kenedy County,NaN,NaN
252,Borden County,NaN,NaN


In [16]:
# 2. Data Cleaning: Covid-19 Social Distancing Score Texas 
social_distancing_scores['County'] = social_distancing_scores['County'].replace(' County', '', regex = True) #Counties have the county name followed by County, used regex to remove the ' County' to be able to merge on this column later.
social_distancing_scores = social_distancing_scores.rename(columns={'Score':'Social_Distancing_Score'})
#social_distancing_scores[social_distancing_scores['Social_Distaning_Score'].isna() == True] There are 10 counties marked as NA, check this against dataframe after to make sure merge worked.
social_distancing_scores

,County,Social_Distancing_Score,Rating
0,Brewster,B-,7.0
1,Fisher,B-,7.0
2,Winkler,B-,7.0
3,Zavala,B-,7.0
4,Throckmorton,B-,7.0
...,...,...,...
249,Terrell,NaN,NaN
250,Loving,NaN,NaN
251,Kenedy,NaN,NaN
252,Borden,NaN,NaN


In [17]:
# 3. Data Transformation: Adding neccessary dummy variables for Social Distancing Scores
social_distancing_scores_dummies = pd.get_dummies(social_distancing_scores, columns=['Social_Distancing_Score'])
social_distancing_scores_dummies

,County,Rating,Social_Distancing_Score_B-,Social_Distancing_Score_C,Social_Distancing_Score_C-,Social_Distancing_Score_D,Social_Distancing_Score_D+,Social_Distancing_Score_D-,Social_Distancing_Score_F
0,Brewster,7.0,1,0,0,0,0,0,0
1,Fisher,7.0,1,0,0,0,0,0,0
2,Winkler,7.0,1,0,0,0,0,0,0
3,Zavala,7.0,1,0,0,0,0,0,0
4,Throckmorton,7.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
249,Terrell,NaN,0,0,0,0,0,0,0
250,Loving,NaN,0,0,0,0,0,0,0
251,Kenedy,NaN,0,0,0,0,0,0,0
252,Borden,NaN,0,0,0,0,0,0,0


## Reading, Cleaning and Transforming Texas Per Capita Income Poverty Rate (NOT Included in Final Model)

In [18]:
# 1. Extract - Reading in the Texas Per Capita Income Poverty Rate data from (https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx?QueryMenuSelectedKeyctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon=944eeb43-ff87-4dc8-bfc2-47d242e9f455)
Texas_Per_Capita_Income_Poverty_Rate = pd.read_excel('Texas_Per_Capita_Income_Poverty_Rate.xlsx')
Texas_Per_Capita_Income_Poverty_Rate

,County,Per Capita Income,Total Personal Income,Median Household Income,Average Annual Pay,% of Population in Poverty,% of Population Under 18 in Poverty
0,Anderson,"$34,242","$1,987,998,000","$45,969","$44,146",19.8,22.6
1,Andrews,"$50,011","$906,592,000","$84,946","$68,340",10.7,14.0
2,Angelina,"$38,897","$3,387,655,000","$46,653","$40,464",17.9,26.7
3,Aransas,"$48,389","$1,151,262,000","$46,912","$38,613",19.9,34.7
4,Archer,"$50,310","$442,022,000","$61,190","$38,231",10.6,14.3
...,...,...,...,...,...,...,...
249,Wood,"$38,152","$1,721,753,000","$48,384","$37,726",15.3,24.9
250,Yoakum,"$41,112","$353,197,000","$61,560","$62,190",12.1,16.3
251,Young,"$50,231","$906,413,000","$49,301","$42,430",13.9,20.4
252,Zapata,"$28,294","$401,494,000","$33,160","$50,175",32.1,49.1


In [19]:
# 2. Data Cleaning: Texas Per Capita Income Poverty Rate
Texas_Per_Capita_Income_Poverty_Rate = Texas_Per_Capita_Income_Poverty_Rate.rename(columns={'Per Capita Income':'Per_Capita_Income','% of Population in Poverty':'Percent_Population_Poverty', 'Median Household Income':'Median_Household_Income'})
Texas_Per_Capita_Income_Poverty_Rate= Texas_Per_Capita_Income_Poverty_Rate.drop(columns= ['Total Personal Income','Average Annual Pay', '% of Population Under 18 in Poverty']) #Removing these columns because the others should suffice, may dip back into some of these at the model progresses. 
Texas_Per_Capita_Income_Poverty_Rate['Per_Capita_Income'] = Texas_Per_Capita_Income_Poverty_Rate['Per_Capita_Income'].replace('[$,]', '', regex = True) # Getting rid of $ and , to be able to process as a float.
Texas_Per_Capita_Income_Poverty_Rate['Median_Household_Income'] = Texas_Per_Capita_Income_Poverty_Rate['Median_Household_Income'].replace('[$,]', '', regex = True)
Texas_Per_Capita_Income_Poverty_Rate

,County,Per_Capita_Income,Median_Household_Income,Percent_Population_Poverty
0,Anderson,34242,45969,19.8
1,Andrews,50011,84946,10.7
2,Angelina,38897,46653,17.9
3,Aransas,48389,46912,19.9
4,Archer,50310,61190,10.6
...,...,...,...,...
249,Wood,38152,48384,15.3
250,Yoakum,41112,61560,12.1
251,Young,50231,49301,13.9
252,Zapata,28294,33160,32.1


## Reading, Cleaning and Transforming Texas Education Levels (NOT Included in Final Model)

In [20]:
# 1. Extract - Reading in the Texas Per Capita Income Poverty Rate data from (https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx?QueryMenuSelectedKeyctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon=a71ea529-f60b-4881-a8d9-a9b16296569c)
Education_Unemployment_Levels = pd.read_excel('Texas_Education_Levels.xlsx')
Education_Unemployment_Levels

,County,% High School Graduate or Higher,% Bachelor s Degree or Higher,Unemployment Rate (%)
0,Anderson,80.2,11.8,3.6
1,Andrews,73.8,10.6,3.1
2,Angelina,79.9,15.7,5.1
3,Aransas,83.2,20.2,6.8
4,Archer,90.1,21.8,3.4
...,...,...,...,...
249,Wood,85.1,17.1,4.7
250,Yoakum,71.4,17.8,3.5
251,Young,81.6,18.6,3.8
252,Zapata,50.3,8.3,7.3


In [21]:
# 2. Data Cleaning: Texas Per Capita Income Poverty Rate
Education_Unemployment_Levels = Education_Unemployment_Levels.rename(columns={'% High School Graduate or Higher': 'Percentage_High_School_Graduate_Or_Higher', '% Bachelor s Degree or Higher': 'Percentage_Bachelors_Degree_Or_Higher','Unemployment Rate (%)':'Unemployment_Rate_Percentage'})
Education_Unemployment_Levels

,County,Percentage_High_School_Graduate_Or_Higher,Percentage_Bachelors_Degree_Or_Higher,Unemployment_Rate_Percentage
0,Anderson,80.2,11.8,3.6
1,Andrews,73.8,10.6,3.1
2,Angelina,79.9,15.7,5.1
3,Aransas,83.2,20.2,6.8
4,Archer,90.1,21.8,3.4
...,...,...,...,...
249,Wood,85.1,17.1,4.7
250,Yoakum,71.4,17.8,3.5
251,Young,81.6,18.6,3.8
252,Zapata,50.3,8.3,7.3


## Reading, Cleaning and Transforming Texas Urban and Rural Splits (Included in Final Model)

In [22]:
# 1. Extract - Reading in the Texas Urban/Rural Data from (https://imis.county.org/iMIS/CountyInformationProgram/QueriesCIP.aspx?QueryMenuSelectedKeyctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon=18d04e71-dba5-4033-a66a-d200b16dda7c)
Rural_Urban = pd.read_excel('Texas_Rural_Urban.xlsx')
Rural_Urban

,County,Land Area,Water Area,Total Area,Percent Urban,Percent Rural
0,Anderson,1062.6,15.4,"1,078.00",32.94,67.06
1,Andrews,1500.7,0.4,"1,501.10",83.50,16.50
2,Angelina,797.8,66.9,864.70,56.92,43.08
3,Aransas,252.1,275.9,528.00,72.74,27.26
4,Archer,903.3,22.3,925.60,11.01,88.99
...,...,...,...,...,...,...
249,Wood,645.2,50.5,695.70,25.80,74.20
250,Yoakum,799.7,0.0,799.70,62.67,37.33
251,Young,914.5,16.4,930.90,66.43,33.57
252,Zapata,998.4,59.6,"1,058.00",76.47,23.53


In [23]:
# 2. Data Cleaning: Texas Urban/Rural Data
Rural_Urban = Rural_Urban.rename(columns={'Percent Urban':'Percent_Urban','Percent Rural':'Percent_Rural', 'Land Area':'Land_Area'})
Rural_Urban= Rural_Urban.drop(columns= ['Water Area','Total Area']) #NEW - adding in land area to calculate population densisty as per sponsors recommendation
Rural_Urban

,County,Land_Area,Percent_Urban,Percent_Rural
0,Anderson,1062.6,32.94,67.06
1,Andrews,1500.7,83.50,16.50
2,Angelina,797.8,56.92,43.08
3,Aransas,252.1,72.74,27.26
4,Archer,903.3,11.01,88.99
...,...,...,...,...
249,Wood,645.2,25.80,74.20
250,Yoakum,799.7,62.67,37.33
251,Young,914.5,66.43,33.57
252,Zapata,998.4,76.47,23.53


## Reading, Cleaning and Transforming Texas Available Resources Data (Included in Final Model)

In [24]:
# 1. Extract - Reading in Available_Resources Data from (https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/0d8bdf9be927459d9cb11b9eaef6101f), 
#Having issues scraping this, code looks like it pings a database. Copied and pasted into excel file instead.
#Raw data copied into Excel then extracted by hand and with VLookup. Matched Trauma Area to County with this resource (https://www.dshs.texas.gov/emstraumasystems/etrarac.shtm: Donwload called Map of Trauma Service Areas with RAC Names)
#This will need to be updated for most up to date availabilty numbers. Current as of 7/26/2020
available_resources = pd.read_excel('Texas_Supplies_Available_Trauma_Area_New_Data.xlsx') 
available_resources['Total_Staffed_Hospital_Beds_Ratio'] = available_resources['Total_Staffed_Hospital_Beds']/available_resources['Trauma_Service_Area_Population_Estimate'] #Creating new series that can keep track of these numbers on a ratio to service area population
available_resources['Available_Hospital_Beds_Ratio'] = available_resources['Available_Hospital_Beds']/available_resources['Trauma_Service_Area_Population_Estimate']
available_resources['Available_ICU_Beds_Ratio'] = available_resources['Available_ICU_Beds']/available_resources['Trauma_Service_Area_Population_Estimate']
available_resources['Available_Ventilators_Ratio'] = available_resources['Available_Ventilators']/available_resources['Trauma_Service_Area_Population_Estimate']
available_resources

,County,Trauma_Area_Letter,Service_Area_RAC,Trauma_Service_Area_Population_Estimate,Total_Staffed_Hospital_Beds,Available_Hospital_Beds,Available_ICU_Beds,Available_Ventilators,Total_Staffed_Hospital_Beds_Ratio,Available_Hospital_Beds_Ratio,Available_ICU_Beds_Ratio,Available_Ventilators_Ratio
0,Armstrong,A,Panhandle_RAC_A,440127,1306,467,68,112,0.002967,0.001061,0.000155,0.000254
1,Briscoe,A,Panhandle_RAC_A,440127,1306,467,68,112,0.002967,0.001061,0.000155,0.000254
2,Carson,A,Panhandle_RAC_A,440127,1306,467,68,112,0.002967,0.001061,0.000155,0.000254
3,Childress,A,Panhandle_RAC_A,440127,1306,467,68,112,0.002967,0.001061,0.000155,0.000254
4,Collingsworth,A,Panhandle_RAC_A,440127,1306,467,68,112,0.002967,0.001061,0.000155,0.000254
...,...,...,...,...,...,...,...,...,...,...,...,...
249,San Patricio,U,Coastal_Bend_RAC_U,630897,2350,1071,148,167,0.003725,0.001698,0.000235,0.000265
250,Cameron,V,Lower_Rio_Grande_Valley_RAC_V,1385112,3307,949,226,181,0.002388,0.000685,0.000163,0.000131
251,Hidalgo,V,Lower_Rio_Grande_Valley_RAC_V,1385112,3307,949,226,181,0.002388,0.000685,0.000163,0.000131
252,Starr,V,Lower_Rio_Grande_Valley_RAC_V,1385112,3307,949,226,181,0.002388,0.000685,0.000163,0.000131


## Reading, Cleaning and Transforming Texas Staffed Beds Per County (NOT Included in Final Model)


In [25]:
# 1. Extract - Reading in staffed beds data from (https://www.ahd.com/states/hospital_TX.html)
#Scraped by importing html file into Excel, counties matched to cities using the match/index functions (see file for details). 
#Data only available for 107 counties (will need to keep this in mind as we progress, will probably not be using this dataset)
#Statistics for non-federal, short-term, acute care hospitals.
staffed_beds = pd.read_excel('Staffed_Beds_County.xlsx')
staffed_beds

,County,Sum of Staffed Beds,Sum of Total Discharges,Sum of Patient Days,Sum of Gross Patient Revenus($000)
0,Anderson County Total,152,3425,19191,399413
1,Andrews County Total,430,21445,87459,2028356
2,Angelina County Total,329,11526,53138,1782438
3,Aransas County Total,1118,50321,258945,7006050
4,Atascosa County Total,67,1711,5288,265251
...,...,...,...,...,...
102,Wichita County Total,291,16022,67591,1195387
103,Willacy County Total,96,4471,11943,137386
104,Wilson County Total,44,790,2831,116248
105,Wise County Total,116,5148,20377,965027


In [26]:
# 2. Data Cleaning: Texas Staffed Beds Data
staffed_beds['County'] = staffed_beds['County'].replace(' County Total', '', regex = True) #Counties have the county name followed by County, used regex to remove the ' County' to be able to merge on this column later.
staffed_beds = staffed_beds.rename(columns={'Sum of Staffed Beds': 'Sum_Staffed_Beds'}) #Renaming value to be able to account for stats work later in R.
staffed_beds = staffed_beds.drop(columns=['Sum of Total Discharges', 'Sum of Patient Days', 'Sum of Gross Patient Revenus($000)']) #Dropping unecessary columns
staffed_beds

,County,Sum_Staffed_Beds
0,Anderson,152
1,Andrews,430
2,Angelina,329
3,Aransas,1118
4,Atascosa,67
...,...,...
102,Wichita,291
103,Willacy,96
104,Wilson,44
105,Wise,116


## Reading, Cleaning and Transforming Texas Medicaid Data (NOT Included in Final Model)


In [27]:
# 1. Extract - Reading in staffed beds data from Texas Medicaid Data from (https://hhs.texas.gov/about-hhs/records-statistics/data-statistics/healthcare-statistics, excel file Medicaid and CHIP Enrollment by Risk Group by County, Preliminary (February 2019, Updated Quarterly))
medicaid_data = pd.read_excel('Medicaid_Data.xlsx', skiprows = 2, skipfooter = 11) # Reading in data and skipping notes/garbage in header and footer.
medicaid_data

,HHSC County Code,County,Medicaid Caseload,Aged & Medicare Related,Disability-Related,Parents*,Pregnant Women,Breast and Cervical Cancer,Children Medicaid**,Medicaid Clients Under 21***,Medicaid Clients 21 and Older,Children's Medicaid**,Regular CHIP,Total
0,1,Anderson,7388.569887,996.009961,956.055780,252.795302,284.125162,7.081217,4892.502464,5202.379628,2186.190259,4892.502464,572.943515,5465.445979
1,2,Andrews,2001.527895,188.406792,151.657294,77.550750,100.397584,3.034807,1480.480668,1572.812446,428.715450,1480.480668,134.751945,1615.232613
2,3,Angelina,14669.112473,1712.974187,2010.483859,389.768055,561.222493,18.208844,9976.455036,10676.446156,3992.666318,9976.455036,1009.125671,10985.580707
3,4,Aransas,3190.087689,348.297962,417.057559,146.037127,112.445294,17.197242,2149.052506,2279.622897,910.464792,2149.052506,249.535994,2398.588501
4,5,Archer,686.875809,86.565283,80.952204,19.135899,23.091444,1.011602,476.119375,507.290384,179.585425,476.119375,72.866401,548.985776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,250,Wood,5481.398711,736.314112,658.889460,245.745234,192.763361,12.139230,3635.547314,3865.170666,1616.228045,3635.547314,433.199272,4068.746586
250,251,Yoakum,1048.663265,108.970415,54.309707,38.271799,57.226623,3.034807,786.849915,836.286240,212.377026,786.849915,74.862058,861.711973
251,252,Young,2436.829937,326.911245,295.116897,83.593666,74.294212,2.023205,1654.890713,1767.026128,669.803809,1654.890713,527.023726,2181.914439
252,253,Zapata,3832.024479,445.047396,287.943917,105.751023,123.489028,5.058012,2864.735104,3047.987412,784.037068,2864.735104,329.387829,3194.122933


In [28]:
# 2. Data Cleaning: Texas Medicaid Data
medicaid_data = medicaid_data.rename(columns={'Medicaid Caseload':'Medicaid_Caseload'}) #Renaming value to be able to account for stats work later in R.
medicaid_data = medicaid_data.drop(columns=['HHSC County Code', 'Aged & Medicare Related','Disability-Related','Parents*','Pregnant Women','Breast and Cervical Cancer','Children Medicaid**','Medicaid Clients Under 21***','Medicaid Clients 21 and Older','Children\'s Medicaid**','Regular CHIP','Total' ]) #Dropping unecessary columns
medicaid_data['Medicaid_Caseload'] = medicaid_data['Medicaid_Caseload'].round()
medicaid_data

,County,Medicaid_Caseload
0,Anderson,7389.0
1,Andrews,2002.0
2,Angelina,14669.0
3,Aransas,3190.0
4,Archer,687.0
...,...,...
249,Wood,5481.0
250,Yoakum,1049.0
251,Young,2437.0
252,Zapata,3832.0


## Reading, Cleaning and Transforming Texas Nursing Home/Assisted Living Data (Included in Final Model)

In [29]:
# 1. Extract - Reading in data of Assisted Living anf Nursing Homes with confirmed cases data found (https://dshs.texas.gov/coronavirus/COVID-19OutbreaksinLong-termCareFacilities.aspx)
Assisted_Living_Nuring_Home_Data = pd.read_excel('Texas COVID-19 Cumulative Facility Totals by Region New Data.xlsx', skipfooter = 1)
Assisted_Living_Nuring_Home_Data

,Public Health Region,Number of Nursing Homes with Confirmed Resident or Staff Cases of COVID-19,Number of Confirmed Cases among Nursing Home Residents,Number of Fatalities among Nursing Home Residents,Number of Reported Recoveriesamong Nursing Home Residents,Number of Assisted Living Facilities with Confirmed Resident or Staff Cases of COVID-19,Number of Confirmed Cases among Assisted Living Facility Residents,Number of Fatalities among Assisted Living Facility Residents,Number of Reported Recoveriesamong Assisted Living Facility Residents
0,1,50,569,96,253,23,47,2,36
1,2/3,261,2959,395,1286,152,450,105,208
2,4/5N,101,1172,153,547,22,28,5,13
3,6/5S,186,3741,437,1313,147,595,68,161
4,7,87,1180,179,515,45,214,31,100
5,8,128,1919,210,554,55,165,33,35
6,9/10,47,1039,146,415,19,135,5,11


In [30]:
# 2. Data Cleaning: Texas Nursing Home and Assisted Living Facility Data
Assisted_Living_Nuring_Home_Data = Assisted_Living_Nuring_Home_Data.drop(columns = ['Number of Reported Recoveriesamong Nursing Home Residents', 'Number of Reported Recoveriesamong Assisted Living Facility Residents', 'Number of Confirmed Cases among Nursing Home Residents','Number of Fatalities among Nursing Home Residents', 'Number of Confirmed Cases among Assisted Living Facility Residents','Number of Fatalities among Assisted Living Facility Residents' ])
Assisted_Living_Nuring_Home_Data = Assisted_Living_Nuring_Home_Data.rename(columns = {'Public Health Region':'Health_Service_Region','Number of Nursing Homes with Confirmed Resident or Staff Cases of COVID-19':'Nursing_Homes_With_Cases_In_HSR', 'Number of Assisted Living Facilities with Confirmed Resident or Staff Cases of COVID-19': 'Assisted_Living_Facilities_With_Cases_In_HSR' })
Assisted_Living_Nuring_Home_Data
#The team may want to calculate this number per health region total population. We will keep this in mind as we continue, it is not something that I think would have a large affect as there are only a few regions.

,Health_Service_Region,Nursing_Homes_With_Cases_In_HSR,Assisted_Living_Facilities_With_Cases_In_HSR
0,1,50,23
1,2/3,261,152
2,4/5N,101,22
3,6/5S,186,147
4,7,87,45
5,8,128,55
6,9/10,47,19


# Reading, Cleaning and Transforming Texas Medicaid Data (NEW Data Pulled After Data Report, (Included in Final Model)) 

In [31]:
uninsured_data = pd.read_excel('Uninsured Data Texas.xlsx') #data from (https://www.countyhealthrankings.org/app/texas/2020/downloads)
uninsured_data #No Cleanup required, taken from excel file exported from the site.
#Uninsured_pop_ratio created after export in excel, could easily implement below once data is merged. 

,County,# Uninsured
0,Anderson,7161
1,Andrews,3486
2,Angelina,15499
3,Aransas,4078
4,Archer,1341
...,...,...
249,Wood,6362
250,Yoakum,1825
251,Young,3281
252,Zapata,3596


## Data Munging/Insertion (also the Load step): Merging Data into a Single Dataframe

In [32]:
data = covid_age_pop.merge(covid_texas,on = 'County', how = 'left') #Merge dataframes
#data[data['Confirmed'].isna() == True] #Checks to make sure the join was successful. 254 counties - 245 in covid data = 9 missing counties which is true. 
data = data.merge(fips_sector_data_dummies, on = 'County', how = 'left') #Merge dataframes
#data[data['White_Pop_Ratio'].isna() == True] #Checks to make sure the join was successful. 254 counties - 245 in covid data = 9 missing counties which is true. 
data = data.merge(covid_race_pop, on = 'County', how = 'left') #Merge dataframes
#data[data['FIPS#'].isna() == True] #Checks to make sure the join was sucessful, no na so it was
data = data.merge(social_distancing_scores_dummies, on = 'County', how = 'left') #Merge dataframes
#data[data['Social_Distaning_Score'].isna() == True]
data = data.merge(Texas_Per_Capita_Income_Poverty_Rate, on = 'County', how = 'left') #Merge dataframes
data = data.merge(Education_Unemployment_Levels, on = 'County', how = 'left') #Merge dataframes
data = data.merge(Rural_Urban, on = 'County', how = 'left') #Merge dataframes
data = data.merge(available_resources, on = 'County', how = 'left') #Merge dataframes
#data[data['Trauma_Area_Letter'].isna() == True]
#data = data.merge(staffed_beds, on = 'County', how = 'left') #Merge dataframes, will probably not be using this as it only has data for 107 counties. 
data = data.merge(medicaid_data, on = 'County', how = 'left') #Merge dataframes
data = data.merge(Assisted_Living_Nuring_Home_Data, on = 'Health_Service_Region', how = 'left') #Merge dataframes
#data[data['Assisted_Living_Facilities_With_Cases_In_PHR'].isna() == True]
data = data.merge(uninsured_data, on = 'County', how = 'left') #Merge dataframes


#adding a few more columns to account for population differences in calculations
#data['Sum_Staffed_Beds_Ratio'] = data['Sum_Staffed_Beds']/data['Population_Estimate']
data['Medicaid_Caseload_Ratio'] = data['Medicaid_Caseload']/data['Population_Estimate']
data['Confirmed_Per_Capita'] = data['Confirmed']/data['Population_Estimate']
data['Deaths_Per_Capita'] = data['Deaths']/data['Population_Estimate']
data['Population_Density'] = data['Population_Estimate']/data['Land_Area']


data['Confirmed'] = data['Confirmed'].fillna(0) #Interting 0s for confirmed as per research explained earlier. Analyzed Texas data and realized blank data from John Hopkins source were supposed to be 0s. 
data['Deaths'] = data['Deaths'].fillna(0) #Interting 0s for confirmed as per research explained earlier. Analyzed Texas data and realized blank data from John Hopkins source were supposed to be 0s.
data['Lethality_Rate'] = data['Lethality_Rate'].fillna(0)

#Have lots of variables to play with here. May add more as we dive deeper into the project and chat about other possible predictors.
#data.to_csv('Covid19_Texas_Data_07_26_all.csv')
data

,County,Population_Estimate,Population_Male,Population_Female,MEDIAN_AGE_TOT,Age017_Per_Pop_Est,Age1844_Per_Pop_Est,Age4554_Per_Pop_Est,Age_5564_Per_Pop_Est,Age_65_plus_Per_Pop_Est,...,Available_ICU_Beds_Ratio,Available_Ventilators_Ratio,Medicaid_Caseload,Nursing_Homes_With_Cases_In_HSR,Assisted_Living_Facilities_With_Cases_In_HSR,# Uninsured,Medicaid_Caseload_Ratio,Confirmed_Per_Capita,Deaths_Per_Capita,Population_Density
0,Anderson,57735,35474,22261,39.4,0.190179,0.400329,0.142461,0.117069,0.149961,...,0.000024,0.000236,7389.0,101.0,22.0,7161,0.127981,0.037066,0.000069,54.333710
1,Andrews,18705,9568,9137,31.5,0.310238,0.377653,0.110131,0.103983,0.097995,...,0.000091,0.000327,2002.0,47.0,19.0,3486,0.107030,0.012029,0.000214,12.464183
2,Angelina,86715,42353,44362,37.5,0.254362,0.333529,0.121651,0.124142,0.166315,...,0.000033,0.000180,14669.0,101.0,22.0,15499,0.169163,0.016203,0.000161,108.692655
3,Aransas,23510,11633,11877,51.2,0.177329,0.254870,0.114334,0.164611,0.288856,...,0.000235,0.000265,3190.0,NaN,NaN,4078,0.135687,0.005232,0.000000,93.256644
4,Archer,8553,4186,4367,44.1,0.215480,0.293581,0.130597,0.164270,0.196072,...,0.000064,0.000136,687.0,261.0,152.0,1341,0.080323,0.001754,0.000000,9.468615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Wood,45539,22602,22937,48.9,0.188564,0.272865,0.109005,0.154483,0.275083,...,0.000024,0.000236,5481.0,101.0,22.0,6362,0.120358,0.005292,0.000132,70.581215
250,Yoakum,8713,4434,4279,30.6,0.326983,0.347871,0.097900,0.108688,0.118558,...,0.000055,0.000401,1049.0,50.0,23.0,1825,0.120395,0.007231,0.000115,10.895336
251,Young,18010,8942,9068,41.6,0.236591,0.298723,0.112937,0.142199,0.209550,...,0.000064,0.000136,2437.0,261.0,152.0,3281,0.135314,0.005608,0.000056,19.693822
252,Zapata,14179,7051,7128,30.0,0.330630,0.345440,0.105367,0.086960,0.131603,...,0.000000,0.000318,3832.0,NaN,NaN,3596,0.270259,0.008604,0.000071,14.201723
